In [8]:
from sedona.spark import *
config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

25/10/20 01:48:14 WARN UDTRegistration: Cannot register UDT for org.geotools.coverage.grid.GridCoverage2D, which is already registered.
25/10/20 01:48:14 WARN SimpleFunctionRegistry: The function rs_union_aggr replaced a previously registered function.
25/10/20 01:48:14 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/10/20 01:48:14 WARN UDTRegistration: Cannot register UDT for org.apache.sedona.common.S2Geography.Geography, which is already registered.
25/10/20 01:48:14 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
25/10/20 01:48:14 WARN SimpleFunctionRegistry: The function st_union_aggr replaced a previously registered function.
25/10/20 01:48:14 WARN SimpleFunctionRegistry: The function st_envelope_aggr replaced a previously registered function.
25/10/20 01:48:14 WARN SimpleFunctionRegistry: The function st_intersection_aggr replaced a previously reg

In [9]:
database = 'gde_silver'

In [10]:
# Home Area Buffers

sedona.sql(f'''
create or replace table org_catalog.{database}.homes_buffers as
select                                                                                                      
sale_id,                                                                                                  
ST_Buffer(geometry, 1600, true) as buffer_1mile
from org_catalog.gde_bronze.king_co_homes 
''')

DataFrame[]

In [11]:
# Flood Hazards Table

sedona.sql(f'''
create or replace table org_catalog.{database}.homes_demographics
with overlaps as (
select                                                                                                      
a.sale_id,   
b.total_pop,
b.median_age,
b.median_income,
st_area(
    st_intersection(a.buffer_1mile, b.geometry)
) / st_area(b.geometry) as share                                                                                             
from org_catalog.{database}.homes_buffers a 
join org_catalog.{database}.census_data b  
on st_intersects(b.geometry, a.buffer_1mile)
),
weighted AS (
  SELECT
    sale_id,
    (total_pop * share) AS pop_in_overlap,  
    median_age,
    median_income
  FROM overlaps
)
SELECT
  sale_id,
  SUM(pop_in_overlap) AS est_total_pop,
  SUM(median_income * pop_in_overlap) / NULLIF(SUM(pop_in_overlap), 0) AS pw_median_income_proxy,
  SUM(median_age * pop_in_overlap) / NULLIF(SUM(pop_in_overlap), 0) AS pw_median_age_proxy
FROM weighted
GROUP BY sale_id
''')

DataFrame[]

In [18]:
# Zoning Percentages

sedona.sql(f'''
create or replace table org_catalog.{database}.zoning_overlaps as
select                                                                                                      
a.sale_id,   
b.MASTER_CAT,
b.SUB_CAT,
st_area(
    st_intersection(a.buffer_1mile, b.geometry)
) / st_area(b.geometry) as share                                                                                             
from org_catalog.{database}.homes_buffers a 
join org_catalog.gde_bronze.gen_land_use_bronze b  
on st_intersects(b.geometry, a.buffer_1mile)
where st_isvalid(geometry) is true
''')

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 3) / 3]
Traceback (most recent call last):
  File "/opt/conda/envs/wherobots/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/wherobots/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/wherobots/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

25/10/20 03:10:11 ERROR AppendDataExec: Data source write support IcebergBatchWrite(table=org_catalog.gde_silver.zoning_overlaps, format=PARQUET) is aborting.
25/10/20 03:10:11 WARN SparkWrite: Skipping cleanup of written files
25/10/20 03:10:11 ERROR AppendDataExec: Data source write support IcebergBatchWrite(table=org_catalog.gde_silver.zoning_overlaps, format=PARQUET) aborted.
25/10/20 03:10:11 ERROR Utils: Aborting task
org.apache.spark.SparkException: Job 25 cancelled as part of cancellation of all jobs
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2731)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$doCancelAllJobs$2(DAGScheduler.scala:1114)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.scala:18)
	at scala.collection.mutable.HashSet$Node.foreach(HashSet.scala:435)
	at scala.collection.mutable.HashSet.foreach(HashSet.sc

In [ ]:
sedona.sql(f'''
create or replace table org_catalog.{database}.homes_zoning_overlaps as
SELECT
  sale_id,
  SUM(CASE WHEN master_cat = 'Industrial' THEN share ELSE 0.0 END) AS industrial,
  SUM(CASE WHEN sub_cat = 'Residential (12+ Units/Acre)' THEN share ELSE 0.0 END) AS urban_residential,
  SUM(CASE WHEN sub_cat = 'Mixed Use' THEN share ELSE 0.0 END) AS mixed_use,
  SUM(CASE WHEN sub_cat = 'Commercial/Office' THEN share ELSE 0.0 END) AS commercial,
  SUM(CASE WHEN master_cat = 'Active Open Space and Recreation' THEN share ELSE 0.0 END) AS recreation,
  SUM(CASE WHEN master_cat = 'Urban Character Residential' THEN share ELSE 0.0 END) AS light_urban,
  SUM(CASE WHEN master_cat = 'Rural Character Residential' THEN share ELSE 0.0 END) AS rural
FROM overlaps
GROUP BY sale_id
''')